#Question 1
The random variable X and Y have the following joint probability density

$f_{XY}(x,y) = \begin{cases}
e^{-x-y}, & 0<x<\infty,0<y<\infty \\
0,& elsewhere \\
\end{cases}$$
$

What is 𝑃 (𝑋 < 𝑌 )? 

##Solution:

\begin{align}
P(X < Y) &= \int_{0}^{\infty}\int_{0}^{y} e^{-x-y} \,dxdy \\
         &= \int_{0}^{\infty}e^{-y}\int_{0}^{y} e^{-x} \,dxdy \\
         &= \int_{0}^{\infty}e^{-y} (1-e^{-y}) \,dy \\
         &= \int_{0}^{\infty}e^{-y} -e^{-2y} \,dy \\
         &= \frac{1}{2} e^{-2y} - e^{-y}\big|_{0}^{\infty} \\
         &= \frac{1}{2}
\end{align}

#Question 2

Counting the pairs with k different from an integer list \\
eg: list = [1, 3,5] and k = 2 \\
expected: we will have 2 pairs: {(1,3), (3,5)} \\
Note: we also consider the negative numbers.

In [1]:
def countPair(nums:list,k:int)->int:
  num_dict = dict()
  for num in nums:
      if num in num_dict:
          num_dict[num] += 1
      else:
          num_dict[num] = 1
  
  count = 0
  if k == 0:
      for key in num_dict:
          if num_dict[key] > 1:
              count += 1
  else:
      for key in num_dict:
          if key+k in num_dict:
              count += 1
  
  return count

In [2]:
nums = [-1,-2,3,4,1,5,7,-3]
k = -4
countPair(nums,k)

4

#Question 3

Return the list of indices. The indices is a sublist points to the same person. The same persons means they have the same name or email or phone.      


eg:



$$
\begin{align}
data = [       &\\
&("username1","phone\_number1", "email1"), \\
&("usernameX","phone\_number1", "emailX"), \\
&("usernameZ","phone\_numberZ", "emailZ"), \\
&("usernameY","phone\_numberY", "emailX"),\\
         ]
\end{align}
$$


expected: $[[0,1,3],[2]]$

In [3]:
def findSamePerson(data):
  import pandas as pd
  import networkx as nx
  from networkx.algorithms.components.connected import connected_components

  #According to each column, find the index of same person
  col=['UserName','PhoneNumber','Email']
  df = pd.DataFrame(data,columns= col).reset_index()
  samePerson = []
  for colName in col:
    samePerson.extend(df.groupby(colName)['index'].apply(list))

  #Merge the lists with the same element
  #Create a graph. Each record is a node, if 2 records have the same field, add an edge
  G = nx.Graph()
  G.add_nodes_from(range(len(data)))
  for ele in samePerson:
    nx.add_path(G,ele)

  #Return the list of connected componets of the Graph
  ret = []
  for person in connected_components(G):
    ret.append(list(person))
  return ret


In [4]:
data = [    
("username1","phone_number1", "email1"),
("usernameX","phone_number1", "emailX"),
("usernameZ","phone_numberZ", "emailZ"),
("usernameY","phone_numberY", "emailX"),
]

findSamePerson(data)


[[0, 1, 3], [2]]

#Question 4

Implement the Forward propagation & Backward propagation for a three layers Neural Network. X,W and b can be random.

##Solution:
- Assume the Loss Function is 

  $ E = \frac{1}{2}(y_{i} - \hat{y_{i}})^2$, then
  $\frac{\partial E}{\partial\hat{y_{i}}} = - y_{i} +\hat{y_{i}}$.

- Assume the activation function is the sigmoid function: 

  $\hat{y_{i}} = \sigma (z)$, then
  $\frac{\partial\hat{y_{i}}}{\partial z} = \sigma(z)*(1-\sigma(z))$

- Assume the integration function is the sum integration function: 

  $ z =w_0x_0+w_1x_1 +w_2x_2 + ...+w_nx_n$ where $w_0 = b$ and $x_0 = 1$,  then
  $\frac{\partial z}{\partial w_{i}} = x_i$

### Update the parameters in Backpropagation :

To update the parameters:
\begin{align}
w_{t+1} = w_{t} - \lambda\frac{\partial E}{\partial w_{t}}
\end{align}

Update parameters between hidden layer node $h_i$ and the output layer node $\hat{y_i}$:


\begin{align}
\frac{\partial E}{\partial w_i^{(2)}}& = \frac{\partial E}{\partial\hat{y_{i}}}
                                \frac{\partial\hat{y_{i}}}{\partial z_i^{(2)}}
                               \frac{\partial z_i^{(2)}}{\partial w_i^{(2)}} \\
                                & = (\hat{y_i} - y_i)\sigma(z_i^{(2)})(1-\sigma(z_i^{(2)}))h_i
\end{align}

Update parameters between input layer node $x_i$ and the hidden layer node $h_i$:


\begin{align}
\frac{\partial E}{\partial w_i^{(1)}}& = \frac{\partial E}{\partial\hat{y_{i}}}
                               \frac{\partial\hat{y_{i}}}{\partial z_i^{(2)}}
                              \frac{\partial z_i^{(2)}}{\partial h_i}
                              \frac {\partial h_i}{\partial z_i^{(1)}}
                              \frac{\partial z_i^{(1)}}{\partial w_i^{(1)}}\\
            & = (\hat{y_i} - y_i)\sigma(z_i^{(2)})(1-\sigma(z_i^{(2)}))w_i^{(2)}\sigma(z_i^{(1)})(1-\sigma(z_i^{(1)}))x_i
\end{align}



In [5]:
def sigmoid(X):
  return 1/(1+np.exp(-X))

def derivativeSigmoid(X):
  return sigmoid(X) * (1-sigmoid(X))

In [6]:
import numpy as np
class threesLayerNN():
  """
  Three Layers: input layer, hidden layer, output layer
  For the hidden layer:
    Assume the integration function = W.T * X + b 
    Assume the activation function = sigmoid
  """
  def __init__(self,n_input:int,n_hidden:int,n_output:int, learning_rate:float):
    self.n_input = n_input
    self.n_hidden = n_hidden
    self.n_output = n_output
    self.inputs = np.zeros(n_input)
    self.integrates_1 = np.zeros(n_hidden)
    self.hiddens = np.zeros(n_hidden)
    self.integrates_2 = np.zeros(n_output)
    self.outputs = np.zeros(n_output)
    self.coefficient_w = [np.random.rand(n_input+1,n_hidden),np.random.rand(n_hidden+1,n_output)]
    self.lr = learning_rate
    #self.intergate_func

  def forwardPropagation(self,input_list):
    if len(input_list) != self.n_input:
      print('The number of input is %d, you are giving %d inputs.' %(self.n_input, len(input_list) ))
      return
    self.inputs = np.array(input_list)
    self.integrates_1 = np.dot(self.coefficient_w[0].T,np.array([1]+list(self.inputs))) #shape = (n_hiddens,)
    self.hiddens = sigmoid(self.integrates_1)
    self.integrates_2 = np.dot(self.coefficient_w[1].T,np.array([1]+list(self.hiddens))) #shape = (n_outputs,)
    self.outputs = sigmoid(self.integrates_2)
    return self.outputs

  def backwardPropagation(self,ground_truth):
    if len(ground_truth) != self.n_output:
      print('The number of outputs is %d, you are giving %d ground thruth.' %(self.n_output, len(ground_truth) ))
      return
    #loss = 0.5*(np.array(ground_truth) - self.outputs)**2
    temp = (self.outputs - np.array(ground_truth)) * derivativeSigmoid(self.integrates_2) #(n_outputs,)

    #Gradient of w between hidden layer and output
    delta_w1 = np.matmul(np.array([1]+list(self.hiddens)).reshape((self.n_hidden+1,1)),temp.reshape((1,self.n_output)))
    #print(delta_w1.shape)

    #Gradient of w between input layer and hidden layer
    a = np.dot(self.coefficient_w[1][1:],temp)*derivativeSigmoid(self.hiddens) #(n_hiddens,)
    delta_w0 =np.matmul(np.array([1]+list(self.inputs)).reshape((self.n_input+1,1)), a.reshape((1,self.n_hidden)))
    #print(delta_w0.shape)

    self.coefficient_w[0] -= self.lr * delta_w0
    self.coefficient_w[1] -= self.lr * delta_w1
    return self.coefficient_w

    
    

In [7]:
t = threesLayerNN(2,4,3,0.5)
for i in range(100):
  t.forwardPropagation([1,2])
  t.backwardPropagation([0.8,0.5,0.8])
t.forwardPropagation([1,2])

array([0.80032668, 0.5000088 , 0.80269432])